# Library inclinometers
This library transforms raw data from inclinometers measurements into a cleaned database.  Some functions aiming a graphical check of measurements are also available.

**Inputs:**

*   a csv-file `'df_rawdata_inklino.csv'` containing the inclinometers' measurements. This file is constructed by hand starting from the raw data obtained by the instruments.

**Outputs:**

*   the multi-index dataframe `df_inklino`. The primary keys of the dataframe are the tuples $(date,tube,depth)$. The tuples $(date,tube)$ allow to access to the empirical data and to the computed deformation / displacement, for a given date and sheet pile type (single or grouted).

In [1]:
#hide
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

# working directory: drive/MyDrive/spundsteif
%cd '/content/drive/MyDrive/spundsteif'

# install dependency (nbdev version 1 and patch tornado problem)
!pip3 install --upgrade notebook
!pip3 install -e . -q
!pip3 install nbdev==1.2.11 -q

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/spundsteif
  Preparing metadata (setup.py) ... done


In [2]:
# default_exp inclinometers

# Import the libraries

In [3]:
#export
import pandas as pd
import numpy
import io
import csv
from datetime import datetime, date

from google.colab import files

from copy import copy

from datetime import timedelta
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go

import numpy as np

# Import the data

In [4]:
#export
def construct_df_csv(file_name):
  return pd.read_csv(file_name, delimiter=';')

# Clean the data

In order to clean the raw inclinometers data, several steps are necessary: useless columns are dropped, a few columns obtain a clearer name, and the dataframe is transformed into a multi-index dataframe.


In [5]:
#export
def drop_columns(df):
  """ Erase the useless columns from the raw data file """

  to_drop = ['format_version',
             'site',
            #  'casing',
             'latitude', 'longitude', 'type',
             'direction', 'mode',
            #  'runs',
              'unit', 'step', 'length', 'azimuth',
              'site_description', 'tube_description', 'os', 'manufacturer', 'model',
              'type2', 'version', 'type3', 'serial', 'firmware', 'T', 'H', 'B', 'V',
              'P', 'serial4', 'firmware5', 'hardware', 'unit6', 'factor',
              'calibration', 'number', 'reference',
            #  'date',
              'delay', 'notes',
            #  'type7',
              'steps', 'number8',
            #  'depth',
              #  'A', 'B9',
              #  'T10',
              'H11', 'V12', 'S']

  df = df.drop(to_drop, 1)

  return df

In [6]:
#export
def update_type_date(df):

  str_days = [date.split('T')[0] for date in df['date']] # We keep only the days

  dateFormatter = "%Y-%m-%d"

  df['date'] = [datetime.strptime(day, dateFormatter) for day in str_days]

  return df

In [7]:
#export
def rename_columns(df):
  df = df.rename(columns={'B9': 'B',
                          'T10': 'temp',
                          'casing': 'tube'})
  return df

In [8]:
#export
def add_column_up(df):
  #valeur + <--> A1B1
  df['info value'] = numpy.where(df['type7'] == 'A1B1', '+', '-')
  df = df.drop(['type7','runs'], 1)
  return df

In [9]:
#export
def df_multiindex(df):
  df_pivot = df.pivot(index=['date','tube','depth'], columns=['info value'],
                      values=['A', 'B'])
  return df_pivot

# Compute rotations and deformations

## Compute average measures

In [10]:
#export
def compute_average_A_B(df):
  df[('A', 'average')] = (df['A']['+'] - df['A']['-']) / 2.
  df[('B', 'average')] = (df['B']['+'] - df['B']['-']) / 2.

  df = df.sort_index(axis=1)

  return df

## Compute rotations

In [11]:
#export
def compute_rotations(df, konstant_inklino = 20000):

  # some inclinometers have a different device constant.
  # 20'000 is the constant of our device.
  #konstant_inklino = 20000.

  df[('A','rotation')] = df[('A', 'average')] / konstant_inklino
  df[('B','rotation')] = df[('B', 'average')] / konstant_inklino

  df = df.sort_index(axis=1)

  return df

## Compute deformations

In [12]:
#export
def __deformee__(list_incl):
  """
  Retourne la liste du calcul de la déformée dans le sens (depth=0, depth=11.5)
  """
  w = 0
  list_w = []

  for theta in list_incl[::-1]:
    w += .5 * np.sin(theta)
    list_w.append(w)

  list_w.reverse()

  return list_w

In [13]:
# export
def column_integration(df):
  df = df.sort_index(axis=0)
  # Pour s'assurer que les hauteurs coincident avec les valeurs de defo ajoutées

  idx = pd.IndexSlice

  # Parcourir pour les couple (date, tube)
  elts_index = df.index.levels
  dates, tubes = elts_index[0], elts_index[1]

  for date in dates:
    for tube in tubes:
      df.loc[idx[date, tube], ('A','defo')] = __deformee__(df.loc[idx[date, tube], ('A','rotation')])
      df.loc[idx[date, tube], ('B','defo')] = __deformee__(df.loc[idx[date, tube], ('B','rotation')])

  df = df.sort_index(axis=1)

  return df

## Compute relative displacements

The user must define the 'start_date'. This date corresponds to the measurement start or to any other, arbitrarily chosen date. All measurements before the start date are erased and the displacements are computed relative to that date.

In [14]:
#export
def get_zero_measure_date(dates, start_date):
  """
  Given a start date returns the date of the measure closest in the past.
  dates: list of sorted datetetime.dates
  start_date: datetime.date
  """

  null_date = min(dates)

  for date in dates:
    if date <= start_date:
      null_date = date

  return null_date

In [15]:
#export
def column_deformee_relative(df, start_date):
  """
  Add a column "relative deformations". The reference value is the start_date
  All measures before this date are deleted
  """

  # Pour s'assurer que les hauteurs coincident avec les valeurs de defo ajoutées
  # et évite les alertes de performance
  df = df.sort_index(axis=0)

  idx = pd.IndexSlice

  elts_index = df.index.levels
  dates, tubes = elts_index[0], elts_index[1]

  # getting null measure date
  date_0 = get_zero_measure_date(dates, start_date)

  nullmessungen_A = {}
  nullmessungen_B = {}

  # setting relative displacements columns
  # A priori la df sera toujours à deux tubes I1 et I2 mais permet de prevenir les cp
  for tube in tubes:
    nullmessungen_A[tube] = numpy.array(df.loc[idx[date_0, tube], ('A','defo')])
    nullmessungen_B[tube] = numpy.array(df.loc[idx[date_0, tube], ('B','defo')])

  for date in dates:
    for tube in tubes:
      df.loc[idx[date, tube], ('A','defo relat')] = numpy.array(df.loc[idx[date, tube], ('A','defo')]) - nullmessungen_A[tube]
      df.loc[idx[date, tube], ('B','defo relat')] = numpy.array(df.loc[idx[date, tube], ('B','defo')]) - nullmessungen_B[tube]

    # Remove measures before null measure
  df = df.truncate(before=date_0)

  return df

The following function '`run_computation_inklino(inklino_file_name, start_date)`' groups all the previously defined functions.

In [16]:
#export
def run_computation_inklino(inklino_file_name, start_date):
  """
  Fonction de computation des mesures inclino
  """
  # Importation sous df du fichier
  df_inklino = construct_df_csv(inklino_file_name)

  # Fonctions de traitement
  df_inklino = drop_columns(df_inklino)
  df_inklino = update_type_date(df_inklino)
  df_inklino = rename_columns(df_inklino)

  df_inklino = add_column_up(df_inklino)

  # Fonction pivot
  df_inklino = df_multiindex(df_inklino)

  # compute rotations and deformations
  df_inklino = compute_average_A_B(df_inklino)
  df_inklino = compute_rotations(df_inklino)

  # Add relative deformations
  df_inklino = column_integration(df_inklino)
  df_inklino = column_deformee_relative(df_inklino, start_date)

  return df_inklino

# Create graphics

In [17]:
# export
def extract_df_datetube(df, date, tube):
  idx = pd.IndexSlice
  return df.loc[idx[date, tube], :]

In [18]:
# export
def build_fig_defo(df, lettre='A'):

  fig = go.Figure()

  elts_index = df.index.levels
  dates, _ = elts_index[0], elts_index[1]

  x_min, x_max = 0, 0
  # Add traces, one for each slider step
  for date in dates:
    df_1 = extract_df_datetube(df, date, 'I1')
    df_2 = extract_df_datetube(df, date, 'I2')

    display_date = date.strftime("%d/%m/%Y")

    fig.add_trace(
        go.Scatter(
          visible=False,
          mode='lines+markers',
          line=dict(color="#D32F2F", width=1),
          name="Unverklebt",
          x=df_1[(lettre,'defo relat')],
          y=df_1.index    # depth
        ))
    fig.add_trace(
        go.Scatter(
          visible=False,
          mode='lines+markers',
          line=dict(color="#4DD0E1", width=1),
          name="Vorveklebt",
          x=df_2[(lettre,'defo relat')],
          y=df_2.index    # depth
        ))

    # Paramètres d'affichage
    x_min = min(min(df_1[(lettre,'defo relat')]),
                min(df_2[(lettre,'defo relat')]),
                x_min
                )

    x_max = max(max(df_1[(lettre,'defo relat')]),
                max(df_2[(lettre,'defo relat')]),
                x_max
                )

  x_max = max(x_max, abs(x_min))
  # Make 1st trace visible
  fig.data[0].visible = True
  fig.data[1].visible = True

  # fig data est une liste de plot / on souhaite tracer les graphs 2 à 2

  # Create and add slider
  steps = []
  for i in range(len(fig.data)//2):
      date_essai = dates[i]
      display_date = date_essai.strftime("%d/%m/%Y")
      step = dict(
          method="update",
          args=[{"visible": [False] * len(fig.data)},
                {"title": "Datum: " + display_date}],  # layout attribute
      )
      step["args"][0]["visible"][2*i] = True  # Toggle i'th trace to "visible"
      step["args"][0]["visible"][2*i+1] = True
      steps.append(step)

  # step = {'method': 'update', 'args': [{'visible': [True, False, False, False]},
  #                                       {'title': 'Slider switched to date: 0'}]}

  sliders = [dict(
      active=0,
      currentvalue={"prefix": "Step: "},
      pad={"t": 50}, # Espace slider graph
      steps=steps
  )]

  # Mise en page
  fig.update_layout(
      width=600,
      height=700,
      sliders=sliders,
      margin=dict(l=40, r=30, t=30, b=30),
      template='none',
      title_font=dict(family="Rockwell", size=18),
      legend=dict(
        orientation="h",
        yanchor="top",
        y=0.93,
        xanchor="center",
        x=0.5
      ),
  )

  fig.update_xaxes(
      domain=(0.2, 0.8),
      range=[-x_max*1.1, x_max*1.1],
      title='Bewegung ' + lettre + ' [m]'
  )

  fig.update_yaxes(
      domain=(0, 0.8),
      range=[12, 0],
      title='Tiefe [m]'
      )

  return fig

# Data analysis Alpnachstad

In the following section, the deformations of the sheet pile wall in Alpnachstad are represented graphically.

*   The displacement in direction A is perpendicular to the sheet pile wall. It's also called **lateral ddisplacement**.
*   The displacement in direction B lies in the sheet pile wall plane. It is also called **transversal displacement**.

**Note:** For some reason, when converting this Jupyter Notebook to a Pdf file, the interactive plots are entirely omitted. These figures can be requested at jean-baptiste.payeur@bfh.ch.

In [19]:
#test_flag_colab
INKLINO = './data/df_rawdata_inklino.csv'
df_inklino = run_computation_inklino(INKLINO, date(2021,7,7))

<ipython-input-5-b3a6b5b0c405>:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(to_drop, 1)
<ipython-input-8-52683cd35b04>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['type7','runs'], 1)
<ipython-input-14-e77b37641806>:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if date <= start_date:


In [20]:
#test_flag_colab
#@title *Lateral displacements*
fig = build_fig_defo(df_inklino, 'A')
fig.show()

In [21]:
#test_flag_colab
#@title *Transversal displacements*
fig = build_fig_defo(df_inklino, 'B')
fig.show()

In [22]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 01_Werkhofversuche_Dehnmessstreifen.ipynb.
Converted 02_inclinometers.ipynb.
Converted 05_catman.ipynb.
Converted 06_strain_gauges.ipynb.
Converted 07_strut_and_earth_pressure.ipynb.
Converted 08_plaxis_linear.ipynb.
Converted 08_plaxis_polynomial.ipynb.
Converted 09_piezometer.ipynb.
Converted Spundsteif_Analyse_essais_flexion.ipynb.
No export destination, ignored:
#export
import pandas as pd
import numpy as np
import io
import csv
from datetime import datetime, date

from google.colab import files

from copy import copy

from datetime import timedelta
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go

from spundsteif import inclinometers # we need also our own libraries
Converted Wylerfeld_inclino.ipynb.
Converted index.ipynb.
